In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

%matplotlib inline

In [2]:
dataset = datasets.MNIST('data', train = True) #without converting to tensor or keepi PIL format

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
dataset[0][0]

In [4]:
#data augmentation and normalisation
stats = ((0.1307 ), (0.3081)) #these will be used as mean and std dev of mnist data

train_transformation = transforms.Compose([transforms.RandomCrop(28,padding=4,padding_mode='reflect'),
                                           transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                           transforms.Normalize(*stats,inplace=True)])

val_transformation = transforms.Compose([transforms.ToTensor(),transforms.Normalize(*stats)])


In [5]:
dataset[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x254E4E57C10>, 5)

In [6]:
train_ds, val_ds = random_split(dataset, [50000,10000])

In [7]:
train_ds[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x254E4E57A90>, 8)

In [8]:
class MapDataset(torch.utils.data.Dataset):
    """
    Given a dataset, creates a dataset which applies a mapping function
    to its items (lazily, only when an item is called).

    Note that data is not cloned/copied from the initial dataset.
    """

    def __init__(self, dataset, map_fn):
        self.dataset = dataset
        self.map = map_fn

#     def __getitem__(self, index):
#         return self.map(self.dataset[index])
    
    def __getitem__(self, index):
        if self.map:     
            x = self.map(self.dataset[index][0]) 
        else:     
            x = self.dataset[index][0]  # image
        y = self.dataset[index][1]   # label      
        return x, y

    def __len__(self):
        return len(self.dataset)

### transform fucntion only works with PIL images

In [9]:
train_sds = MapDataset(train_ds, train_transformation)

In [10]:
val_sds = MapDataset(val_ds, val_transformation)

In [11]:
len(val_sds)

10000

In [12]:
image,label = train_sds[0]
image.shape

torch.Size([1, 28, 28])

In [13]:
train_sds[5666][1]

8

In [14]:
type(train_sds[5666])

tuple

In [15]:
train_loader = DataLoader(train_sds, batch_size=128, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_sds, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)

# now next

In [16]:
# def main():
#     for i,l in val_loader:
#         print(i.shape)
#         print(l.shape)
#         break

# if __name__ == '__main__':
#     main()

In [17]:
import workers
main = workers.main

In [18]:
if __name__ == '__main__':
    main(val_loader)

RuntimeError: DataLoader worker (pid(s) 6744) exited unexpectedly